# Classification of Night Sky Objects

As a warm up, lets create some classifiers to predict whether an object is star, galaxy, or qso purely based on the light we receive.  Using redshift would be a dead give away, so we'll ignore that.

In [74]:
# Load in libraries
import os
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.dummy import DummyRegressor, DummyClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.tree import DecisionTreeRegressor, export_graphviz, DecisionTreeClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

%matplotlib inline

In [4]:
# Set working directory
os.chdir(os.path.join(sys.path[0], ".."))

In [85]:
# Read in data
training_data = pd.read_csv(os.path.join(".", "data", "processed", "total", "train.csv"))
test_data = pd.read_csv(os.path.join(".", "data", "processed", "total", "test.csv"))

In [86]:
# Check out the data
print(f"Size of Training Data: {training_data.shape}")
print(f"Distribution of Objects in Training Data:\n{training_data['class'].value_counts() / training_data.shape[0]}")
print("")
print(f"Size of Training Data: {test_data.shape}")
print(f"Distribution of Objects in Test Data:\n{test_data['class'].value_counts() / test_data.shape[0]}")

Size of Training Data: (80000, 19)
Distribution of Objects in Training Data:
GALAXY    0.594812
STAR      0.215638
QSO       0.189550
Name: class, dtype: float64

Size of Training Data: (20000, 19)
Distribution of Objects in Test Data:
GALAXY    0.59300
STAR      0.21715
QSO       0.18985
Name: class, dtype: float64


In [87]:
# Check out data
training_data.head()

,Unnamed: 0,obj_ID,alpha,delta,u,g,r,i,z,run_ID,rerun_ID,cam_col,field_ID,spec_obj_ID,redshift,plate,MJD,fiber_ID,class
0,75220,1.237661e+18,155.057478,39.471934,22.32247,21.40113,20.51302,19.62691,19.34791,3704,301,6,151,5.136368e+18,0.659869,4562,55570,45,GALAXY
1,48955,1.237679e+18,29.186923,33.102558,25.85486,24.22573,21.48514,20.38426,19.38826,7781,301,5,116,8.709046e+18,0.813819,7735,58136,763,GALAXY
2,44966,1.237668e+18,143.780304,16.857367,22.62750,21.41766,19.54710,18.87770,18.39861,5183,301,2,88,5.986547e+18,0.404277,5317,56000,499,GALAXY
3,13568,1.237665e+18,154.753807,34.679618,20.25937,18.53387,17.58141,17.16960,16.85448,4518,301,6,179,2.200026e+18,0.108840,1954,53357,63,GALAXY
4,92727,1.237662e+18,212.519940,36.429773,22.03892,21.46501,21.32925,21.26120,21.17124,3900,301,3,521,4.340534e+18,-0.001039,3855,55268,690,STAR


In [89]:
# Encode target
training_data['encoded_class']=training_data['class'].astype('category').cat.codes
test_data['encoded_class']=test_data['class'].astype('category').cat.codes

In [90]:
# Separate out variables for modeling
drop_features = ["Unnamed: 0", "obj_ID", "alpha", "delta", "run_ID", "rerun_ID",
                 "cam_col", "field_ID", "spec_obj_ID", "redshift",
                 "plate", "MJD", "fiber_ID", "class"]
target = "encoded_class"


numeric_features = list(
    set(training_data.columns)
    - set([target])
    - set(drop_features)
)
assert training_data.columns.shape[0] == len(
    drop_features + numeric_features + [target]
)

In [91]:
# Create transformer
preprocessor = make_column_transformer(
    (StandardScaler(), numeric_features),
    ("drop", drop_features),
)

In [92]:
# Separate data into train test
X_train, y_train = training_data.drop(columns=[target]), training_data[target]
X_test, y_test = test_data.drop(columns=[target]), test_data[target]

In [93]:
# Fit the preprocessor
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('standardscaler', StandardScaler(),
                                 ['z', 'g', 'u', 'i', 'r']),
                                ('drop', 'drop',
                                 ['Unnamed: 0', 'obj_ID', 'alpha', 'delta',
                                  'run_ID', 'rerun_ID', 'cam_col', 'field_ID',
                                  'spec_obj_ID', 'redshift', 'plate', 'MJD',
                                  'fiber_ID', 'class'])])

In [94]:
from sklearn.metrics import make_scorer


def mape(true, pred):
    return 100.0 * np.mean(np.abs((pred - true) / true))


# make a scorer function that we can pass into cross-validation
mape_scorer = make_scorer(mape, greater_is_better=False)

scoring_metrics = {
    "neg RMSE": "neg_root_mean_squared_error",
    "r2": "r2",
    "mape": mape_scorer,
}

In [95]:
results = {}

In [96]:
def mean_std_cross_val_scores(model, X_train, y_train, **kwargs):
    """
    Returns mean and std of cross validation

    Parameters
    ----------
    model :
        scikit-learn model
    X_train : numpy array or pandas DataFrame
        X in the training data
    y_train :
        y in the training data

    Returns
    ----------
        pandas Series with mean scores from cross_validation
    """

    scores = cross_validate(model, X_train, y_train)

    mean_scores = pd.DataFrame(scores).mean()
    std_scores = pd.DataFrame(scores).std()
    out_col = []

    for i in range(len(mean_scores)):
        out_col.append((f"%0.3f (+/- %0.3f)" % (mean_scores[i], std_scores[i])))

    return pd.Series(data=out_col, index=mean_scores.index)


In [97]:
dummy = DummyClassifier(strategy="most_frequent")

scores = cross_validate(dummy, X_train, y_train)

results["Dummy"] = mean_std_cross_val_scores(
    dummy, X_train, y_train, return_train_score=True, scoring=scoring_metrics
)

pd.DataFrame(results)

,Dummy
fit_time,0.007 (+/- 0.000)
score_time,0.001 (+/- 0.000)
test_score,0.595 (+/- 0.000)


In [98]:
dt_pipe = make_pipeline(preprocessor, DecisionTreeClassifier())

results["decisicion tree"] = mean_std_cross_val_scores(
    dt_pipe, X_train, y_train, return_train_score=True, scoring=scoring_metrics
)

pd.DataFrame(results)

,Dummy,decisicion tree
fit_time,0.007 (+/- 0.000),0.677 (+/- 0.011)
score_time,0.001 (+/- 0.000),0.010 (+/- 0.002)
test_score,0.595 (+/- 0.000),0.812 (+/- 0.005)


In [99]:
pipe_rf = make_pipeline(preprocessor, RandomForestClassifier(random_state=42))
pipe_xgb = make_pipeline(preprocessor, XGBClassifier(random_state=42))
pipe_catboost = make_pipeline(
    preprocessor, CatBoostClassifier(verbose=0, random_state=42)
)

models = {
    "random forest": pipe_rf,
    "XGBoost": pipe_xgb,
    "CatBoost": pipe_catboost,
}

In [100]:
for (name, model) in models.items():
    results[name] = mean_std_cross_val_scores(
        model, X_train, y_train, return_train_score=True, scoring=scoring_metrics
    )

pd.DataFrame(results)

,Dummy,decisicion tree,random forest,XGBoost,CatBoost
fit_time,0.007 (+/- 0.000),0.677 (+/- 0.011),17.296 (+/- 0.112),10.244 (+/- 0.379),9.571 (+/- 0.053)
score_time,0.001 (+/- 0.000),0.010 (+/- 0.002),0.384 (+/- 0.006),0.036 (+/- 0.003),0.025 (+/- 0.002)
test_score,0.595 (+/- 0.000),0.812 (+/- 0.005),0.875 (+/- 0.004),0.870 (+/- 0.004),0.868 (+/- 0.003)
